In [2]:
import json

In [3]:
f = open('mpqa_csds.json', 'r', encoding='utf-8')

In [4]:
data = json.load(f)

In [31]:
import re

def replace_punc(clean_text, offset, offset_list):
    poss_dif = 0
    s = clean_text
    s, off1 = re.subn(' \.', '.', s)
    s, off2 = re.subn(' ,', ',', s)
    s, off3 = re.subn(' ;', ';', s)
    s, off4 = re.subn(' :', ':', s)
    s, off5 = re.subn(' POSS', '\'s', s)
    if off5 > 0:
        poss_dif = 3
    adjust_val = off1+off2+off3+off4+poss_dif
    offset -= adjust_val
    if (off1+off2+off3+off4+poss_dif > 0):
        offset_list[-1] -= adjust_val
    return s, offset, offset_list

In [5]:
def assemble_tokens(w_text):
    clean_text = ''
    offset = 0
    offset_list = [0]
    index = 0
    in_quote = False
    while index < len(w_text):
        if (index == 0):
            clean_text += w_text[index]
            offset += len(w_text[index])
            index+=1
            continue
        if w_text[index] == '"' or w_text[index] == '\'':
            if not in_quote:
                clean_text += ' ' + w_text[index] + w_text[index+1]
                offset += 1
                offset_list.append(offset)
                offset+= len(w_text[index])
                offset_list.append(offset)
                #for the word itself
                index+=1
                offset += len(w_text[index])
                in_quote = True
            else:
                clean_text += w_text[index]
                offset_list.append(offset)
                offset += 1
                in_quote = False
        else:
            clean_text += ' '
            offset += 1
            offset_list.append(offset)
            clean_text += w_text[index]    
            offset += len(w_text[index])
        clean_text, offset, offset_list = replace_punc(clean_text, offset, offset_list)
        index += 1
    return clean_text, offset_list

In [32]:
limit = 100
count = 0
punc_list = ['.', ',', ';', ':', '\"', '\'']
for obj in data:
    if count > limit:
        break
    s1 = set(['w_head_span', 'w_text', 'clean_head', 'head_start', 'head_end'])
    for ind in range(len(obj['w_text'])):
        if obj['w_text'][ind] == '\'s':
            obj['w_text'][ind] = "POSS"
    if s1.issubset(obj.keys()):
        clean, offset_list = assemble_tokens(obj['w_text'])
        w_head, w_end = obj['w_head_span']
        # print(clean)
        print('ORIGINAL:', obj['clean_head'])
        s_head = offset_list[w_head]
        e_head = offset_list[w_end]
        if w_head == 0 and w_end == 0:
            print('     NEW:', '')
        elif offset_list[-1] == e_head or clean[offset_list[w_end+1]-2] in punc_list or clean[offset_list[w_end+1]-1] in punc_list:
            print('     NEW:', clean[s_head:e_head])
        else:
            print('     NEW:', clean[s_head:e_head-1])
    count+=1

ORIGINAL: branded as "evil "
     NEW: branded as "evil"
ORIGINAL: "nuclear club "
     NEW: "nuclear club"
ORIGINAL: may have erred
     NEW: may have erred
ORIGINAL: more clearly
     NEW: more clearly
ORIGINAL: real
     NEW: real
ORIGINAL: as far as human rights are concerned
     NEW: as far as human rights are concerned
ORIGINAL: easily qualify as repressive and violating human rights
     NEW: easily qualify as repressive and violating human rights
ORIGINAL: oppressive
     NEW: oppressive
ORIGINAL: not in itself
     NEW: not in itself
ORIGINAL: aggressively
     NEW: aggressively
ORIGINAL: evildoer
     NEW: evildoer
ORIGINAL: therefore
     NEW: therefore
ORIGINAL: scores of
     NEW: scores of
ORIGINAL: seemingly
     NEW: seemingly
ORIGINAL: Perhaps
     NEW: Perhaps
ORIGINAL: In other words
     NEW: In other words
ORIGINAL: aggressively
     NEW: aggressively
ORIGINAL: grave human rights violators
     NEW: grave human rights violators
ORIGINAL: irrespective
     NEW: irr